In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Download a pretrained model from tensorflow model zoo
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
!mv efficientdet_d0_coco17_tpu-32.tar.gz data/models/
!rm efficientdet_d0_coco17_tpu-32.tar.gz

In [ ]:
# load data from google drive for training
!unzip -q ../../gdrive/My\ Drive/oivd_data.zip -d  ../

In [ ]:
!pip install pycocotools

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import tensorflow as tf
from object_detection import model_lib_v2

In [ ]:
#Path to pipeline config file.
pipeline_config_path=None
#Number of train steps.
num_train_steps=10
#Enable evaluating on train data (only supported in distributed training).
eval_on_train_data=False
#Will sample one of every n eval input examples, where n is provided.
sample_1_of_n_eval_examples= None
#Will sample one of every n train input examples for evaluation, where n is provided. This is only used if `eval_training_data` is True.
sample_1_of_n_eval_on_train_examples=5
#where event and checkpoint files will be written.
model_dir=None
#Path to directory holding a checkpoint.  If `checkpoint_dir` is provided, this binary operates in eval-only mode, writing resulting metrics to `model_dir`.
checkpoint_dir=None
#Number of seconds to wait for an evaluation checkpoint before exiting.
eval_timeout=3600
#Whether the job is executing on a TPU.
use_tpu=False
#Name of the Cloud TPU for Cluster Resolvers. TPU is automatically inferred if tpu_name is None and if running under cloud ai-platform.
tpu_name=None
#When num_workers > 1, training uses MultiWorkerMirroredStrategy. When num_workers = 1 it uses MirroredStrategy.
num_workers=1
#Integer defining how often we checkpoint.
checkpoint_every_n=1000
#Whether or not to record summaries during training.
record_summaries=True

In [ ]:
tf.config.set_soft_device_placement(True)

In [ ]:
if checkpoint_dir:
    model_lib_v2.eval_continuously(
        pipeline_config_path=pipeline_config_path,
        model_dir=model_dir,
        train_steps=num_train_steps,
        sample_1_of_n_eval_examples=sample_1_of_n_eval_examples,
        sample_1_of_n_eval_on_train_examples=(
            sample_1_of_n_eval_on_train_examples),
        checkpoint_dir=checkpoint_dir,
        wait_interval=300, timeout=eval_timeout)
else:
    if use_tpu:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu_name)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
    elif num_workers > 1:
        strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    else:
        strategy = tf.compat.v2.distribute.MirroredStrategy()

    with strategy.scope():
        model_lib_v2.train_loop(
            pipeline_config_path=pipeline_config_path,
            model_dir=model_dir,
            train_steps=num_train_steps,
            use_tpu=use_tpu,
            checkpoint_every_n=checkpoint_every_n,
            record_summaries=record_summaries)

In [ ]:
!unzip ../../gdrive/My\ Drive/oivd_data.zip -d  ../

In [ ]:
import shutil

In [ ]:
cp -r ../../gdrive/My\ Drive/images ../

In [ ]:
temp_path=os.getcwd()

In [ ]:
os.chdir("../")

In [ ]:
path="OID/Dataset"
for dirpath,dirnames,filenames in os.walk(os.path.join(path,"train")):
    for filename in filenames:
        shutil.copy(os.path.join(dirpath,filename),os.path.join("images","train"))

for dirpath,dirnames,filenames in os.walk(os.path.join(path,"test")):
    for filename in filenames:
        shutil.copy(os.path.join(dirpath,filename),os.path.join("images","test"))

In [ ]:
!mv images/train survillence-system/data/images
!mv images/test survillence-system/data/images

In [ ]:
os.chdir(temp_path)

In [ ]:
import xml.etree.ElementTree as ET
import glob

path="data/images"
for xml_file in glob.glob(path +"/train" + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            text_to_save=root.find('filename').text.split(".")[0]
            root.find('filename').text=text_to_save
        tree.write(xml_file)

for xml_file in glob.glob(path +"/test" + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            text_to_save=root.find('filename').text.split(".")[0]
            root.find('filename').text=text_to_save
        tree.write(xml_file)

In [ ]:
!python create_detection_data.py --classes=data/labels/animaltheft.names

In [ ]:
!python convert.py

In [ ]:
!python convert.py --tiny --output=models/checkpoints/yolov3_tiny/yolov3_tiny.tf --weights=weights/yolov3-tiny.weights

In [ ]:
!ls models/checkpoints

In [ ]:
!python train.py --transfer=darknet --classes=data/labels/animaltheft.names --epochs=100 --batch_size=32 --weights=models/checkpoints/yolov3/yolov3.tf --output=models/checkpoints/theft --num_classes=3 --weights_num_classes=80

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!tensorboard --logdir=logs --host 0.0.0.0

In [ ]:
!ls models/checkpoints

In [ ]:
!ls models/checkpoints/theft/

In [ ]:
!zip val_best_theft.zip models/checkpoints/theft/yolov3_train_16.tf.index models/checkpoints/theft/yolov3_train_16.tf.data-00000-of-00002 models/checkpoints/theft/yolov3_train_16.tf.data-00001-of-00002

In [ ]:
# !zip loss_best_theft.zip models/checkpoints/theft/yolov3_train_60.tf.index models/checkpoints/theft/yolov3_train_60.tf.data-00000-of-00002 models/checkpoints/theft/yolov3_train_60.tf.data-00001-of-00002

In [ ]:
# !mv loss_best_theft.zip ../../gdrive/My\ Drive/theft_model_loss_best_frozen.zip
!mv val_best_theft.zip ../../gdrive/My\ Drive/theft_model_val_best.zip

In [ ]:
!rm -rf models/checkpoints/theft/*
!rm -rf ~/.local/share/Trash/*

In [ ]:
!python train.py --tiny --transfer=darknet --classes=data/labels/animaltheft.names --epochs=100 --batch_size=32 --weights=models/checkpoints/yolov3_tiny/yolov3_tiny.tf --output=models/checkpoints/theft_tiny --num_classes=3 --weights_num_classes=80

In [ ]:
!ls models/checkpoints/theft_tiny/

In [ ]:
!ls models/checkpoints

In [ ]:
!zip val_best_theft_tiny.zip models/checkpoints/theft_tiny/yolov3_train_9.tf.index models/checkpoints/theft_tiny/yolov3_train_9.tf.data-00000-of-00002 models/checkpoints/theft_tiny/yolov3_train_9.tf.data-00001-of-00002

In [ ]:
# !zip loss_best_theft_tiny.zip models/checkpoints/theft_tiny/yolov3_train_29.tf.index models/checkpoints/theft_tiny/yolov3_train_29.tf.data-00000-of-00002 models/checkpoints/theft_tiny/yolov3_train_29.tf.data-00001-of-00002

In [ ]:
# !mv loss_best_theft_tiny.zip ../../gdrive/My\ Drive/theft_tiny_model_loss_best.zip
!mv val_best_theft_tiny.zip ../../gdrive/My\ Drive/theft_tiny_model_val_best.zip

In [ ]:
!rm -rf models/checkpoints/theft_tiny/*
!rm -rf ~/.local/share/Trash/*